In [14]:
import pandas as pd

# load csv file
file_path = '/home/valerio/git/superpoint_transformer_vschelbi/logs/partition_parametrization_4.0.csv'
df = pd.read_csv(file_path)


def filter_level_ratios(df):
    return df[
        (df["|P_0| / |P_1|"] > 20) & # normally above 30
        (df["|P_1| / |P_2|"] > 2) # normally above 3
    ]

# other filters TODO

# filter the dataframe to contain only parametrizations that satisfy level ratio constraints
level_ratio_filtered = filter_level_ratios(df)
# group by 'param_set_id'
grouped_df = level_ratio_filtered.groupby('param_set_id')
valid_param_set_ids = [name for name, group in grouped_df if len(group) == 5]
filtered_df = df[df['param_set_id'].isin(valid_param_set_ids)]

In [15]:
len(filtered_df) / 5  # number of valid parametrizations

16.0

In [16]:
numeric_cols = filtered_df.select_dtypes(include='number').columns.difference(['param_set_id']) # get all numeric columns except 'param_set_id'

# calculate the mean metrics of all 5 datasets for each valid parametrization
mean_metrics_df = filtered_df.groupby('param_set_id')[numeric_cols].mean().reset_index()

In [17]:
# add the 'param_set' column to the mean_metrics_df based on the 'param_set_id'
mean_metrics_df = mean_metrics_df.merge(
    filtered_df[['param_set_id', 'param_set']].drop_duplicates(),
    on='param_set_id')

desired_order = [col for col in filtered_df.columns if col in mean_metrics_df.columns]

mean_metrics_df = mean_metrics_df[desired_order]
mean_metrics_df.to_csv('/home/valerio/git/superpoint_transformer_vschelbi/logs/mean_metrics_open_ended_levelratios_parametrization_4.0.csv', index=True)
